In [3]:
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
import nltk
nltk.download('punkt')

documents = [
  '''
  This is the document
  '''
]

# Preprocess the documents
def preprocess(text):
    # Tokenize and remove stopwords
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

processed_docs = [preprocess(doc) for doc in documents]

# Create a dictionary from the processed documents
dictionary = corpora.Dictionary(processed_docs)

# Create a corpus (bag of words)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Train the LDA model
num_topics = 2  # You can adjust this number based on your needs
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Print the topics
print("Topics:")
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")

# # Infer topics for a new document
# new_doc = "This is the title"
# bow_vector = dictionary.doc2bow(preprocess(new_doc))
# for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
#     print(f"Score: {score:.4f} Topic: {lda_model.print_topic(index, 5)}")

Topics:
Topic 0: 1.000*"document"
Topic 1: 1.000*"document"


In [ ]:
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
import nltk
nltk.download('punkt')

# Sample documents (replace with your own corpus)
documents = [
  '''
  optimus prime vs sentinal prime
  '''
]

# Preprocess the documents
def preprocess(text):
    # Tokenize and remove stopwords
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

processed_docs = [preprocess(doc) for doc in documents]

# Create a dictionary from the processed documents
dictionary = corpora.Dictionary(processed_docs)

# Create a corpus (bag of words)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Train the LDA model
num_topics = 2  # You can adjust this number based on your needs
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42)

# Print the topics
print("Topics:")
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")

# Infer topics for a new document
new_doc = "Artificial intelligence and machine learning are transforming various industries"
bow_vector = dictionary.doc2bow(preprocess(new_doc))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print(f"Score: {score:.4f} Topic: {lda_model.print_topic(index, 5)}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Topics:
Topic 0: 0.300*"prime" + 0.241*"sentinal" + 0.231*"vs" + 0.229*"optimus"
Topic 1: 0.351*"prime" + 0.219*"optimus" + 0.218*"vs" + 0.212*"sentinal"
Score: 0.5000 Topic: 0.300*"prime" + 0.241*"sentinal" + 0.231*"vs" + 0.229*"optimus"
Score: 0.5000 Topic: 0.351*"prime" + 0.219*"optimus" + 0.218*"vs" + 0.212*"sentinal"


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embedding(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

    # Get the embeddings for the text
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings of the [CLS] token (the first token)
    embedding = outputs.last_hidden_state[0][0]
    return embedding

def compute_cosine_similarity(embedding1, embedding2):
    return torch.nn.functional.cosine_similarity(embedding1, embedding2, dim=0)

def compute_keyword_overlap(topic, essay):
    vectorizer = CountVectorizer().fit([topic, essay])
    vectors = vectorizer.transform([topic, essay])
    overlap = np.dot(vectors[0].toarray(), vectors[1].toarray().T)[0][0]
    return overlap / (np.linalg.norm(vectors[0].toarray()) * np.linalg.norm(vectors[1].toarray()))

def compute_similarity(topic, essay):
    # Get embeddings for topic and essay
    topic_embedding = get_embedding(topic)
    essay_embedding = get_embedding(essay)

    # Compute cosine similarity between the embeddings
    cosine_similarity = compute_cosine_similarity(topic_embedding, essay_embedding).item()

    # Compute keyword overlap
    keyword_overlap = compute_keyword_overlap(topic, essay)

    # Combine cosine similarity and keyword overlap into a final score
    combined_score = (cosine_similarity + keyword_overlap) / 2

    # Scale similarity score to a range of 0 to 100
    score = (combined_score + 1) * 50
    return score

# Example usage
topic1 = "The impact of artificial intelligence on modern society"
essay1 = "Artificial intelligence has revolutionized various aspects of our lives. From smart assistants to autonomous vehicles, AI is reshaping how we interact with technology. However, this rapid advancement also raises ethical concerns and questions about job displacement. As AI continues to evolve, it's crucial to consider both its benefits and potential drawbacks on society."

topic2 = "The impact of porn on modern society"
essay2 = "Artificial intelligence has revolutionized various aspects of our lives. From smart assistants to autonomous vehicles, AI is reshaping how we interact with technology. However, this rapid advancement also raises ethical concerns and questions about job displacement. As AI continues to evolve, it's crucial to consider both its benefits and potential drawbacks on society."

topic3 = "The impact of porn on modern society"
essay3 = "Artificial intelligence has revolutionized various aspects of our lives. From smart assistants to autonomous vehicles, AI is reshaping how we interact with technology. However, this rapid advancement also raises ethical concerns and questions about job displacement. As AI continues to evolve, it's crucial to consider both its benefits and potential drawbacks on society."

score1 = compute_similarity(topic1, essay1)
score2 = compute_similarity(topic2, essay2)
score3 = compute_similarity(topic3, essay3)

print(f"Score1: {score1:.2f}")
print(f"Score2: {score2:.2f}")
print(f"Score2: {score2:.2f}")

Score1: 73.41
Score2: 71.22
Score2: 70.86


In [6]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [58]:
from sentence_transformers import SentenceTransformer, util

query = "The importance of urban green spaces"
docs = ["Urban green spaces are vital for the health and well-being of city dwellers. Parks, gardens, and tree-lined streets provide a much-needed respite from the concrete jungle, offering areas for relaxation, exercise, and social interaction. These natural havens improve air quality by filtering pollutants and producing oxygen. They also help mitigate the urban heat island effect, reducing temperatures in densely populated areas. Green spaces support biodiversity, providing habitats for various plant and animal species within urban environments. Moreover, they contribute to mental health by reducing stress and promoting mindfulness. As cities continue to grow, it's crucial that urban planners prioritize the preservation and creation of green spaces to ensure livable, sustainable cities for future generations." , "Upload your speech audio file to get a raw transcription, pronunciation and fluency scores of your speech. This tool uses advanced speech recognition to transcribe your audio and evaluate pronunciation quality. Note that the translations that it provides are raw and is exactly what an average person hears of. The misspelled words in the transcription are not fixed because this raw translation is important for your scoring", "in order to change i have to be something or not i came here to turn things around to become the best in the world unless id beat some one stronger than me that loa a ma a anan ban man m ana ban a man anan bo"]

#Load the model
model = SentenceTransformer('sentence-transformers/msmarco-distilbert-cos-v5')

In [59]:
docs = "urban"

In [60]:
#Encode query and documents
query_emb = model.encode(query)
doc_emb = model.encode(docs)

In [61]:
query_emb.shape

(768,)

In [62]:
doc_emb.shape

(768,)

In [63]:
query_emb

array([-1.53356036e-02, -1.37274200e-02,  2.66055632e-02,  3.59833948e-02,
        3.04549467e-02,  1.50385695e-02, -6.10584393e-02,  2.23194994e-02,
       -7.18197748e-02, -3.25868987e-02, -3.83623727e-02, -6.42792508e-02,
       -6.37580175e-03,  2.59747100e-03,  2.44397167e-02,  1.31479204e-02,
       -2.60523502e-02, -1.63934790e-02,  3.21543473e-03,  6.48022303e-03,
        1.69414934e-02, -5.33828959e-02, -2.93269400e-02,  2.95225102e-02,
        4.27778736e-02, -1.70110837e-02, -3.29770036e-02,  3.19257677e-02,
       -3.40852663e-02,  2.78953817e-02,  8.51008762e-03, -6.69292547e-03,
       -4.27724347e-02,  2.45844759e-02,  5.25137968e-02,  1.90054823e-03,
        1.35711511e-03, -5.34118935e-02, -6.45285770e-02,  2.43115909e-02,
       -7.73120113e-03,  6.63633123e-02,  4.05159825e-03,  1.38615891e-02,
        2.32236423e-02,  2.85917222e-02, -7.97701478e-02, -8.49186815e-03,
        2.70489417e-02, -1.61854289e-02,  4.39286381e-02, -3.55898738e-02,
       -4.12222221e-02,  

In [64]:
doc_emb

array([-1.87318232e-02, -3.18142697e-02,  2.59631276e-02,  2.78667901e-02,
       -4.13650833e-02,  4.59331535e-02,  2.22657025e-02, -4.47335020e-02,
       -3.40617672e-02, -4.72656935e-02, -7.03940913e-02,  5.04460000e-02,
       -3.50616276e-02, -3.59882601e-02,  7.14938939e-02,  2.70737364e-04,
       -3.24144810e-02, -1.45383691e-02, -1.84519049e-02,  1.38294157e-02,
       -1.64624080e-02, -4.89476323e-02, -2.22984375e-03,  8.30052048e-03,
        2.14315969e-02, -2.17034314e-02, -3.98699827e-02, -1.72426365e-02,
       -4.25605960e-02, -3.13715748e-02, -1.37965009e-02,  1.56429578e-02,
       -2.03292537e-02,  2.36197226e-02, -8.94780084e-03, -6.00778796e-02,
       -1.45274447e-02, -8.35073516e-02, -3.68755125e-02,  3.15170512e-02,
        1.20429951e-03,  1.83582678e-02, -1.69655159e-02,  4.27599065e-03,
       -7.68524874e-03, -4.68180329e-02, -3.24824937e-02, -5.88846356e-02,
       -2.64195222e-02,  4.32756431e-02,  3.44513953e-02,  1.10061988e-02,
       -4.28371206e-02,  

In [65]:
#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb).cpu()[0][0]

In [67]:
float(scores)

0.48255735635757446

In [ ]:
#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
for doc, score in doc_score_pairs:
    print(score, doc)